### 模型搭建

In [2]:
config_path = '../News Sentiment classification/data/model/chinese_wwm_ext_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../News Sentiment classification/data/model/chinese_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../News Sentiment classification/data/model/chinese_wwm_ext_L-12_H-768_A-12/vocab.txt'

In [3]:
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [4]:
class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]')
            else:
                R.append('[UNK]')
        return R

In [5]:
tokenizer = OurTokenizer(token_dict)

In [6]:
def acc_top2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)
def build_bert(n_class):
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

    for l in bert_model.layers:
        l.trainable = True

    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))

    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x) # 获得[CLS]位置的向量表示
    p = Dense(n_class, activation='softmax')(x)

    model = Model([x1_in, x2_in], p)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(1e-5), # 用足够小的学习率
        metrics=['accuracy', acc_top2]
    )
    print(model.summary())
    return model

### 数据处理

In [7]:
train_df = pd.read_csv('./data/new_train.csv', names=['id', 'r_title', 'r_desc', 'a_title', 'a_desc', 'label'])
test_df = pd.read_csv('./data/new_test.csv', names=['id', 'r_title', 'r_desc', 'a_title', 'a_desc', 'none'])

In [8]:
maxlen = 200 # 单句最大长度

In [9]:
def seq_padding(X, Padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [Padding]* (ML - len(x))]) if len(x) < ML else x for x in X
    ])

In [10]:
from keras.utils import to_categorical

DATA_LIST = []
for data_row in train_df.iloc[:].itertuples():
    DATA_LIST.append((data_row.r_title + data_row.r_desc,
                      data_row.a_title + data_row.a_desc,
                      to_categorical(data_row.label-1, 4).astype(np.int32)))
DATA_LIST = np.array(DATA_LIST)

DATA_LIST_TEST = []
for data_row in test_df.iloc[:].itertuples():
    DATA_LIST_TEST.append((data_row.r_title + data_row.r_desc,
                           data_row.a_title + data_row.a_desc,
                           to_categorical(0, 4).astype(np.int32)))
DATA_LIST_TEST = np.array(DATA_LIST_TEST)

In [11]:
class data_generator:
    def __init__(self, data, batch_size=2, shuffle=True):
        self.data = data #上面的numpy数组
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data))) # data大小
            
            if self.shuffle:
                np.random.shuffle(idxs)
            
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                requirement = d[0][:maxlen]
                achievement = d[1][:maxlen]
                x1, x2 = tokenizer.encode(first=requirement, second=achievement)
                y = d[2]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y[:, 0, :]
                    [X1, X2, Y] = [], [], []

### 训练

In [14]:
def run_model(data, data_label, data_test):
    train_pred = np.zeros((len(data), 4))
    test_pred = np.zeros((len(data_test), 4))
    
    # 划分验证集
    random_order = range(len(data))
    np.random.shuffle(list(random_order))
    X_train = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
    X_valid = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]
    
    model = build_bert(4)
    early_stopping = EarlyStopping(monitor='val_acc', patience=3)
    checkpoint = ModelCheckpoint('bert_dump/ach_re' + '.hdf5', monitor='val_acc', 
                                         verbose=2, save_best_only=True, mode='max',save_weights_only=True)
    # 学习率衰减
    plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
    
    # 数据生成器
    train_D = data_generator(X_train, shuffle=True)
    valid_D = data_generator(X_valid, shuffle=True)
    test_D = data_generator(data_test, shuffle=False)
        
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=5,
        validation_data=valid_D.__iter__(),
        validation_steps=len(valid_D),
        callbacks=[early_stopping, plateau, checkpoint],
    )
    train_pred[test_fold, :] =  model.predict_generator(valid_D.__iter__(), steps=len(valid_D),verbose=1)
    test_pred += model.predict_generator(test_D.__iter__(), steps=len(test_D),verbose=1)
    
    # 清除内存与session
    del model
    gc.collect()
    K.clear_session()
    
    return train_pred, test_pred

In [15]:
train_model_pred, test_model_pred = run_model(DATA_LIST, None, DATA_LIST_TEST)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, None, 768)    101677056   input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 768)          0           model_5[1][0]                    
__________

NameError: name 'maxlen' is not defined